In [31]:
import spacy as spacy
#check https://towardsdatascience.com/auto-generated-knowledge-graphs-92ca99a81121
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy
import re
import os.path
my_path = os.path.abspath('')

In [32]:
#read all the text file in the directory
def file_reader(path):
    files = []
    for r,d,f in os.walk(path):
        for file in f:
            if '.txt' in file:
                files.append(os.path.join(r,file))       
    return files

#print elements from list
def print_list(lists):
    for list in lists:
        print(list)

In [33]:
nlp = spacy.load('en_core_web_lg')


files = file_reader(my_path+"/clean_data/item1_business")
files.sort()
print_list(files)




/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_0_section_1.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_10_section_0.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_11_section_0.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_13_section_0.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_14_section_0.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_15_section_0.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_16_section_0.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_bu

In [158]:
text=open(files[2])
#print(text.read())
#if \n Blablab \n if the first character is capitalized, add period
#if \n blablab \n if the first character is not capitalized, remove the first \n

In [159]:

doc = nlp(text.read())
options = {"ents": ["PERSON", "ORG", "PRODUCT"]}
#displacy.render(doc, style="ent", jupyter= True, options=options)

In [160]:
#find competitors
for sent in doc.sents:
    check=0
    relations = list()
    for token in sent:
        if token.lemma_ =="competitor" or token.lemma_ =="compete" or token.lemma_ =="competition":
                #entities_relation.append(token.text)
            if check==0 and len(sent.ents)!=0: displacy.render(sent, style="ent", jupyter= True, options=options)
            relations.append(token.text)
            check=check+1
            #displacy.render(sent, style="dep", jupyter= True, options={"distance":140, "collapse_phrases":True})
    if len(relations)>0 and len(sent.ents)!=0:print(relations)

['competes']


['compete']


['competed']


['competition']


['competition']


['competition']


In [161]:
#find products and services
for sent in doc.sents:
    check=0
    relations = list()
    for token in sent:
        if token.lemma_ =="product" or token.lemma_ =="service" or token.lemma_ =="sell" or token.lemma_=="device" or token.lemma_=="provide" or token.lemma_=="offer" or token.lemma_=="develop" or token.lemma_=="design" or token.lemma_ =="manufacture" or token.lemma_ =="tool":
            if check==0 and len(sent.ents)!=0: displacy.render(sent, style="ent", jupyter= True, options=options)
            relations.append(token.text)
            check=check+1
            
    if len(relations)>0 and len(sent.ents)!=0:print(relations)

['provides', 'services']


['Products', 'offered']


['services']


['services']


['providing', 'products', 'products']


...

['services']


['sold']


['services']


['offers', 'provides']


['provide']


In [162]:
#find customer
for sent in doc.sents:
    check=0
    relations = list()
    for token in sent:
        if  token.lemma_=="client" or token.lemma_=="manufacturer" or token.lemma_=="customer" or token.lemma_=="buyer" or token.lemma_=="purchase":
            if check==0 and len(sent.ents)!=0: displacy.render(sent, style="ent", jupyter= True, options=options)
            relations.append(token.text)
            check=check+1
    if len(relations)>0 and len(sent.ents)!=0:print(relations)

['clients', 'clients']


['clients']


['clients']


['Clients']


['clients', 'clients']


...

['clients']


['customers']


['clients']


['clients']


['customer']


In [163]:
#find m&a activities
for sent in doc.sents:
    check=0
    relations = list()
    for token in sent:
        if token.lemma_=="acquisition" or token.lemma_=="acquire" or token.lemma_=="merger" or token.lemma_=="merge" or token.lemma_=="consolidate" or token.lemma_=="consolidation":
            if check==0 and len(sent.ents)!=0: displacy.render(sent, style="ent", jupyter= True, options=options)
            relations.append(token.text)
            check=check+1
            
    if len(relations)>0 and len(sent.ents)!=0:print(relations)

['acquired']


['acquired', 'acquisition', 'acquired', 'acquired']


['acquisitions']


['acquired']


['acquired']


['acquired']


['consolidation']


In [164]:
#try to find sector
for sent in doc.sents:
    check=0
    for token in sent:
        if token.lemma_ =="industry" or token.lemma_ =="sector":
                #entities_relation.append(token.text)
            if check==0: displacy.render(sent, style="ent", jupyter= True, options=options)
            check=check+1
            print("["+token.text+"]")
            #displacy.render(sent, style="dep", jupyter= True, options={"distance":140, "collapse_phrases":True})
            print()

/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


[industry]



[industry]



[industry]



[industry]



...

[sector]



[industry]



[industry]



[industry]

[industry]



[industry]

[sector]



In [141]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

# Patterns are lists of dictionaries describing the tokens
pattern1 = [{"LEMMA": "product", "POS": "PROPN"}]
pattern2 = [{"LEMMA": "product", "POS": "NOUN"}]
matcher.add("products_sent", None, pattern1)

In [ ]:

#print entities NER
entity_pairs = []
for sent in doc.sents:
    if sent.ents and (e.label_=="ORG" or e.label_=="PERSON" or e.label_=="PRODUCT" for e in doc.ents):
        #entity_pairs.append([sent.ents[0],sent.ents[1]])
        print(sent)
        print(sent.ents)
        #print("["+sent.ents[0].text+", "+sent.root.text+", "+sent.ents[1].text+"]" )
        print()
    #if ent.label_=='ORG' or ent.label_=='PRODUCT' or ent.label_=='PERSON':
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)


In [ ]:
for i in range(len(entity_pairs)):
    print("["+entity_pairs[i][0].text+", "+ entities_relation[i]+", "+ entity_pairs[i][1].text+"]")

In [ ]:
#use pos to get relations
#use pos to get non entities objects

pos_chain = "-".join([d.tag_ for d in sent])
    print(sent)
    print(pos_chain)
    if "NNP-VBZ-NNP" in pos_chain:

In [ ]:
#get verbs from every sentences dep_ as relations
entities_relation = []
for sent in doc.sents:
        for token in sent:
            if token.pos_=="VERB" or token.pos_=="AUX":
                #entities_relation.append(token.text)
                print(sent)
                print("["+token.text+"]")
                print()